In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model

In [3]:
#file=r'https://www.dropbox.com/s/49kyq1xl2g7pp6y/train.csv?dl=0'
#file=r'D:\\Edvancer\\deeplearning\\projects\\P3-spam quora\\train.csv'

'''
Download datasets
'''
!wget https://www.dropbox.com/s/49kyq1xl2g7pp6y/train.csv

--2021-03-31 18:11:02--  https://www.dropbox.com/s/49kyq1xl2g7pp6y/train.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/49kyq1xl2g7pp6y/train.csv [following]
--2021-03-31 18:11:02--  https://www.dropbox.com/s/raw/49kyq1xl2g7pp6y/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc60dfd11932f5de09fd6aae2ea2.dl.dropboxusercontent.com/cd/0/inline/BLuWwglbGQDowqUWoGozmO4--YcK9j3eHQTCIPH4_orOug8_6PUFYo0VjH2ANURSPLCP-GuE0XpYODzESDd640trMHPt1yvygUcH2gdvlFvb3g0d5CdNhVyzER3Kh4k6X0PQXE8SAvoMDHdXLrNtCbbD/file# [following]
--2021-03-31 18:11:03--  https://uc60dfd11932f5de09fd6aae2ea2.dl.dropboxusercontent.com/cd/0/inline/BLuWwglbGQDowqUWoGozmO4--YcK9j3eHQTCIPH4_orOug8_6PUFYo0VjH2ANURSPLCP-GuE0XpYODzESDd640trMHPt1yvygUc

In [4]:
file=r'/content/train.csv'

In [5]:

quora=pd.read_csv(file)

In [6]:
quora

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [7]:
quora_train,quora_test=train_test_split(quora,test_size=0.2,random_state=2)

In [8]:
x_train=quora_train['question_text']
y_train=quora_train['target']
x_test=quora_test['question_text']
y_test=quora_test['target']

In [9]:
x_train.shape

(1044897,)

In [10]:
x_test.shape

(261225,)

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
'''
Here we are finding sentence with max words
'''

sent_lens=[]
for sent in quora_train['question_text']:
    sent_lens.append(len(word_tokenize(sent)))
    
    

In [13]:
max(sent_lens)

412

In [ ]:
# sent_lens

In [14]:
'''
trying to find the 95 % of the sentences are less than equal to no of words
'''
np.quantile(sent_lens,0.95)


30.0

In [ ]:
# We can see that 95% review text are of lengths less than or equal to 30. We'll keep the max length to 30

In [15]:
max_len = 30

tok = Tokenizer(char_level=False,split=' ')

tok.fit_on_texts(x_train)

In [ ]:
#tok.index_word

In [16]:
sequences_train = tok.texts_to_sequences(x_train)

In [ ]:
# sequences_train

In [17]:
word_index=tok.word_index

In [18]:
'''
number of unique words in the x_train
'''
vocab_len=len(tok.index_word.keys())

In [19]:
vocab_len

196201

In [20]:
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len)

In [21]:
sequences_matrix_train

array([[   0,    0,    0, ..., 5916,   44,  242],
       [   0,    0,    0, ...,    5, 1602, 3144],
       [   0,    0,    0, ...,  136,   18,  296],
       ...,
       [   0,    0,    0, ...,  392,    7,  678],
       [   0,    0,    0, ...,    6,   28,  560],
       [   0,    0,    0, ..., 5472,    6,  142]], dtype=int32)

In [22]:
!wget 'http://nlp.stanford.edu/data/glove.6B.zip'
!unzip '/content/glove.6B.zip'

--2021-03-31 18:24:47--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-03-31 18:24:48--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-03-31 18:24:48--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [23]:
'''
Preparing the Embedding layer

Next, we compute an index mapping words to known embeddings, by parsing the data dump of pre-trained embeddings:
'''
import os
embeddings_index = {}
f = open(os.path.join('/content/', r'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [24]:
'''
At this point we can leverage our embedding_index dictionary and our word_index to compute our embedding matrix:
'''
EMBEDDING_DIM=100
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [25]:
'''
We load this embedding matrix into an Embedding layer. Note that we set trainable=False to prevent the weights from being updated during training.
'''

from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)


In [26]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    
    #layer = Embedding(vocab_len+1,500,input_length=max_len, mask_zero=True)(inputs)
    embedded_sequences = embedding_layer(inputs)
    layer = LSTM(64)(embedded_sequences)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model



In [27]:
model = RNN()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 30)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 100)           19620200  
_________________________________________________________________
lstm (LSTM)                  (None, 64)                42240     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257   

In [28]:
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])

In [29]:
sequences_test = tok.texts_to_sequences(x_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,
                                               maxlen=max_len)

In [30]:
model.fit(sequences_matrix_train,y_train.values,batch_size=500,
          epochs=10,validation_data=(sequences_matrix_test,y_test.values))

Epoch 1/10
2090/2090 [==============================] - 46s 7ms/step - loss: 0.1580 - accuracy: 0.9430 - val_loss: 0.1212 - val_accuracy: 0.9525
Epoch 2/10
2090/2090 [==============================] - 15s 7ms/step - loss: 0.1190 - accuracy: 0.9534 - val_loss: 0.1128 - val_accuracy: 0.9554
Epoch 3/10
2090/2090 [==============================] - 15s 7ms/step - loss: 0.1122 - accuracy: 0.9560 - val_loss: 0.1104 - val_accuracy: 0.9563
Epoch 4/10
2090/2090 [==============================] - 15s 7ms/step - loss: 0.1072 - accuracy: 0.9578 - val_loss: 0.1098 - val_accuracy: 0.9567
Epoch 5/10
2090/2090 [==============================] - 15s 7ms/step - loss: 0.1046 - accuracy: 0.9587 - val_loss: 0.1088 - val_accuracy: 0.9572
Epoch 6/10
2090/2090 [==============================] - 15s 7ms/step - loss: 0.1023 - accuracy: 0.9592 - val_loss: 0.1082 - val_accuracy: 0.9572
Epoch 7/10
2090/2090 [==============================] - 15s 7ms/step - loss: 0.0994 - accuracy: 0.9602 - val_loss: 0.1119 - val_ac

In [31]:
predictions=model.predict(sequences_matrix_test)

In [32]:
from sklearn.metrics import roc_auc_score

In [33]:
roc_auc_score(y_test,predictions)

0.958098073295852